## Importing libraries

In [1]:
import sys
sys.path.append('../') 
from imports import *
import cudf
import cupy as cp
import cuml
from cuml import LogisticRegression as logreg

In [2]:
def plot_roc_pr_curves(y_true, y_probs, n_bootstraps=1000):
    """Perform bootstrapping to calculate 95% confidence intervals for ROC and PR curves and plot them"""
    bootstrap_auroc_scores = []
    bootstrap_average_precision_scores = []

    for _ in range(n_bootstraps):
        # Bootstrap sample (with replacement)
        indices = resample(np.arange(len(y_true)), replace=True)
        y_true_boot = y_true[indices]
        y_probs_boot = y_probs[indices]

        # Compute metrics for bootstrap sample
        bootstrap_auroc_scores.append(roc_auc_score(y_true_boot, y_probs_boot))
        bootstrap_average_precision_scores.append(average_precision_score(y_true_boot, y_probs_boot))

    # Compute confidence intervals
    auroc_lower = np.percentile(bootstrap_auroc_scores, 2.5)
    auroc_upper = np.percentile(bootstrap_auroc_scores, 97.5)
    ap_lower = np.percentile(bootstrap_average_precision_scores, 2.5)
    ap_upper = np.percentile(bootstrap_average_precision_scores, 97.5)

    # Calculate original ROC and PR curves
    fpr, tpr, _ = roc_curve(y_true, y_probs)
    precision, recall, _ = precision_recall_curve(y_true, y_probs)
    auroc = roc_auc_score(y_true, y_probs)
    average_precision = average_precision_score(y_true, y_probs)

    # Plotting
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

    # ROC Curve
    ax1.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {auroc:.2f})')
    ax1.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    ax1.set_xlabel('False Positive Rate')
    ax1.set_ylabel('True Positive Rate')
    ax1.set_title('Receiver Operating Characteristic')
    ax1.legend(loc="lower right", title=f'95% CI: [{auroc_lower:.2f}, {auroc_upper:.2f}]')

    # Precision-Recall Curve
    ax2.plot(recall, precision, color='blue', lw=2, label=f'PR curve (area = {average_precision:.2f})')
    ax2.set_xlabel('Recall')
    ax2.set_ylabel('Precision')
    ax2.set_title('Precision-Recall Curve')
    ax2.legend(loc="lower left", title=f'95% CI: [{ap_lower:.2f}, {ap_upper:.2f}]')

    plt.show()

    # Return the confidence intervals
    return (np.round(auroc_lower,2), np.round(auroc_upper,2)), (np.round(ap_lower,2), np.round(ap_upper,2))




# Using whole pipeline on GPU using NVIDIA RAPIDS - do later

## Load pre-processed saved datasets

In [3]:
# Load the saved datasets
X_train = np.load('./fashion-mnist-processed-data/train_data.npy')
X_val = np.load('./fashion-mnist-processed-data/val_data.npy')
X_test = np.load('./fashion-mnist-processed-data/test_data.npy')

y_train = np.load('./fashion-mnist-processed-data/train_targets.npy')
y_val = np.load('./fashion-mnist-processed-data/val_targets.npy')
y_test = np.load('./fashion-mnist-processed-data/test_targets.npy')

In [4]:
# Reshape the training data
n_samples, height, width = X_train.shape
X_train = X_train.reshape((n_samples, height*width))

# Similarly, reshape the validation and test data 
n_val_samples, height_val, width_val = X_val.shape
X_val = X_val.reshape((n_val_samples, height_val*width_val))

n_test_samples, height_test, width_test = X_test.shape
X_test = X_test.reshape((n_test_samples, height_test*width_test))

# convert to cupy arrays
X_train_cp = cp.array(X_train, dtype=cp.float32)
y_train_cp = cp.array(y_train, dtype=cp.float32)
X_val_cp = cp.array(X_val, dtype=cp.float32)
y_val_cp = cp.array(y_val, dtype=cp.float32)
X_test_cp = cp.array(X_test, dtype=cp.float32)
y_test_cp = cp.array(y_test, dtype=cp.float32)

In [5]:
X_cudf = cudf.DataFrame.from_pandas(pd.DataFrame(X_train))

## Logistic Regression without regularization

In [6]:
# Initialize and fit the logistic regression model
log_reg = logreg(penalty='none', output_type='numpy', max_iter=10000)
log_reg.fit(X_train_cp, y_train_cp)

[W] [03:41:06.196077] L-BFGS line search failed (code 3); stopping at the last valid step


LogisticRegression()

In [7]:
# Predict on validation set
y_val_pred = log_reg.predict(X_val)

# Predict probabilities
y_val_probs = log_reg.predict_proba(X_val)

# Evaluate the model
val_accuracy = accuracy_score(y_val, y_val_pred)
print(f"Validation Accuracy: {val_accuracy:.4f}")
print("Validation Classification Report:\n", classification_report(y_val, y_val_pred))

Validation Accuracy: 0.8492
Validation Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.81      0.81       600
           1       0.95      0.96      0.96       600
           2       0.75      0.76      0.75       600
           3       0.87      0.89      0.88       600
           4       0.74      0.78      0.76       600
           5       0.94      0.91      0.93       600
           6       0.65      0.59      0.62       600
           7       0.93      0.92      0.92       600
           8       0.91      0.92      0.92       600
           9       0.91      0.95      0.93       600

    accuracy                           0.85      6000
   macro avg       0.85      0.85      0.85      6000
weighted avg       0.85      0.85      0.85      6000



## Logistic Regression with L1 regularization

In [8]:
# Initialize and fit the logistic regression model
log_reg_l1 = logreg(penalty='l1', output_type='numpy', max_iter=10000)
log_reg_l1.fit(X_train_cp, y_train_cp)

LogisticRegression()

In [9]:
# Predict on validation set
y_val_pred_l1 = log_reg_l1.predict(X_val)

# Predict probabilities
y_val_probs_l1 = log_reg_l1.predict_proba(X_val)

# Evaluate the model
val_accuracy = accuracy_score(y_val, y_val_pred_l1)
print(f"Validation Accuracy: {val_accuracy:.4f}")
print("Validation Classification Report:\n", classification_report(y_val, y_val_pred_l1))

Validation Accuracy: 0.8518
Validation Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.82      0.81       600
           1       0.96      0.97      0.97       600
           2       0.76      0.76      0.76       600
           3       0.86      0.90      0.88       600
           4       0.76      0.77      0.76       600
           5       0.94      0.92      0.93       600
           6       0.64      0.60      0.62       600
           7       0.91      0.92      0.92       600
           8       0.93      0.92      0.93       600
           9       0.93      0.94      0.94       600

    accuracy                           0.85      6000
   macro avg       0.85      0.85      0.85      6000
weighted avg       0.85      0.85      0.85      6000



## Logistic Regression with L2 regularization

In [10]:
# Initialize and fit the logistic regression model
log_reg_l2 = logreg(penalty='l2', output_type='numpy', tol=1e-2, max_iter=10000)
log_reg_l2.fit(X_train_cp, y_train_cp)

LogisticRegression()

In [11]:
# Predict on validation set
y_val_pred_l2 = log_reg_l2.predict(X_val)

# Predict probabilities
y_val_probs_l2 = log_reg_l2.predict_proba(X_val)

# Evaluate the model
val_accuracy = accuracy_score(y_val, y_val_pred_l2)
print(f"Validation Accuracy: {val_accuracy:.4f}")
print("Validation Classification Report:\n", classification_report(y_val, y_val_pred_l2))

Validation Accuracy: 0.8532
Validation Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.82      0.82       600
           1       0.96      0.96      0.96       600
           2       0.75      0.75      0.75       600
           3       0.87      0.90      0.89       600
           4       0.74      0.78      0.76       600
           5       0.94      0.92      0.93       600
           6       0.66      0.58      0.62       600
           7       0.92      0.92      0.92       600
           8       0.93      0.94      0.94       600
           9       0.93      0.95      0.94       600

    accuracy                           0.85      6000
   macro avg       0.85      0.85      0.85      6000
weighted avg       0.85      0.85      0.85      6000



### Hyperparameter tuning for logistic regression with L1 regularization

In [12]:
# Define a set of C values to try
C_values = [0.001, 0.01, 0.1, 1, 10, 100]

# Initialize variables to store the best score and corresponding C value
best_score = 0
best_C = None

# Perform grid search over the C values
for C in C_values:
    # Initialize and train the Logistic Regression model with L1 regularization
    log_reg_l1 = logreg(penalty='l1', C=C, output_type='numpy', tol=1e-2, max_iter=10000)
    log_reg_l1.fit(X_train_cp, y_train_cp)

    # Evaluate on the validation set
    y_val_pred_l1 = log_reg_l1.predict(X_val)
    score = accuracy_score(y_val, y_val_pred_l1)

    # If the score is better than the best score, update the best score and best C
    if score > best_score:
        best_score = score
        best_C = C

# Output the best C value
print(f"Best C value: {best_C} with AUROC: {best_score:.4f}")



Best C value: 0.01 with AUROC: 0.8658


## Final model: Logistic Regression with L1 regularization with C=0.01

In [13]:
# Train a new model using the best C value
best_log_reg_l1 = logreg(penalty='l1', C=best_C, output_type='numpy', max_iter=10000)
best_log_reg_l1.fit(X_train_cp, y_train_cp)

[W] [03:41:29.059967] QWL-QN stopped, because the line search failed to advance (step delta = 0.000000)


LogisticRegression()

In [14]:
# Predict on validation set
y_val_pred_1 = best_log_reg_l1.predict(X_val)

# Predict probabilities
y_val_probs_1 = best_log_reg_l1.predict_proba(X_val)

# Evaluate the model
val_accuracy_1 = accuracy_score(y_val, y_val_pred_1)
print(f"Validation Accuracy: {val_accuracy_1:.4f}")
print("Validation Classification Report:\n", classification_report(y_val, y_val_pred_1))

Validation Accuracy: 0.8612
Validation Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.83      0.82       600
           1       0.97      0.97      0.97       600
           2       0.77      0.77      0.77       600
           3       0.86      0.91      0.89       600
           4       0.76      0.77      0.77       600
           5       0.95      0.94      0.94       600
           6       0.66      0.61      0.63       600
           7       0.92      0.93      0.93       600
           8       0.93      0.94      0.94       600
           9       0.95      0.95      0.95       600

    accuracy                           0.86      6000
   macro avg       0.86      0.86      0.86      6000
weighted avg       0.86      0.86      0.86      6000



In [15]:
# Predict on test set
y_test_pred_l1 = best_log_reg_l1.predict(X_test)

# Predict probabilities
y_test_probs_l1 = best_log_reg_l1.predict_proba(X_test)

# Evaluate the model
test_accuracy = accuracy_score(y_test, y_test_pred_l1)
print(f"Validation Accuracy: {test_accuracy:.4f}")
print("Validation Classification Report:\n", classification_report(y_test, y_test_pred_l1))

Validation Accuracy: 0.8424
Validation Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.81      0.81      1000
           1       0.97      0.95      0.96      1000
           2       0.73      0.75      0.74      1000
           3       0.82      0.86      0.84      1000
           4       0.73      0.76      0.75      1000
           5       0.94      0.92      0.93      1000
           6       0.63      0.56      0.59      1000
           7       0.92      0.94      0.93      1000
           8       0.92      0.93      0.93      1000
           9       0.94      0.94      0.94      1000

    accuracy                           0.84     10000
   macro avg       0.84      0.84      0.84     10000
weighted avg       0.84      0.84      0.84     10000

